In [47]:
import cv2
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd



In [55]:
pth_root = r"C:\workspace\hakaton\dataset"

leb_types = ['klikun', 'maliy', 'shipun']

name2id = {
    'klikun': 0,
    'maliy': 1,
    'shipun': 2
}

id2name = {v: k for k, v in name2id.items()}

print(os.listdir(pth_root))

['klikun', 'maliy', 'shipun', 'Определение_лебедей_по_внешнему_виду.docx', 'Пояснение к оценке.docx']


In [56]:

def find_box_by_mask(image, color):
    """
    Находит координаты бокса объекта на основе маски по цвету
    Возвращает координаты бокса [x, y, x2, y2] или None, если объект не найден
    """
    mask = image == color
    # Находим индексы True значений в маске
    nonzero_indices = np.transpose(np.nonzero(mask))

    # Если нет True значений, объект не найден
    if len(nonzero_indices) == 0:
        return None

    # Находим границы бокса
    min_x = np.min(nonzero_indices[:, 1])
    max_x = np.max(nonzero_indices[:, 1])
    min_y = np.min(nonzero_indices[:, 0])
    max_y = np.max(nonzero_indices[:, 0])

    # Вычисляем координаты бокса
    box = [min_x, min_y, max_x, max_y]

    return box

In [57]:
result_list  = []

uniq_id_frame = 0
for type_name in leb_types:

    pth_masks = os.path.join(pth_root, type_name, 'masks')
    file_list = os.listdir(pth_masks)
    for file_mask in tqdm(file_list):
        base_name = file_mask.split('.')[0]
        pth_img = os.path.join(pth_masks, file_mask)
        img = cv2.imread(pth_img)
        objs = np.unique(img)
        objs.sort()
        objs = objs[1:] # фон не учитываем
        for id_obj, obj_color in enumerate(objs):
            bb = find_box_by_mask(image=img, color=obj_color)
            pth_mask_img = os.path.join('masks', file_mask)
            pth_img = os.path.join('images', base_name + ".jpg")
            h, w, _ = img.shape
            result_list.append((uniq_id_frame, type_name, name2id[type_name], id_obj, pth_img, pth_mask_img, *bb, h, w))
            uniq_id_frame += 1


100%|██████████| 3011/3011 [04:34<00:00, 10.95it/s]


In [58]:
columns = ['frame_id', 'type_name', 'type_id', 'obj_id', 'path_img', 'path_mask', 'x1', 'y1', 'x2', 'y2', 'height', 'width']
df = pd.DataFrame(result_list, columns=columns)
df.to_csv('all_annotation.csv', index=False)

In [73]:
# Составляем список уникальных классов
unique_classes = df['type_name'].unique()

train_df = pd.DataFrame()
test_df = pd.DataFrame()

# Разделяем данные на тренировочный и тестовый наборы для каждого класса
for class_name in unique_classes:
    # Фильтруем данные для текущего класса
    class_data = df[df['type_name'] == class_name]

    # Разделяем данные на тренировочный и тестовый наборы
    train_data, test_data = train_test_split(class_data, test_size=0.15, random_state=42)

    # Добавляем данные текущего класса к тренировочному и тестовому DataFrame
    train_df = pd.concat(train_df.append(train_data))
    test_df = test_df.append(test_data)

C:\Users\annza\AppData\Local\Temp\ipykernel_18016\1122127876.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(train_data)
C:\Users\annza\AppData\Local\Temp\ipykernel_18016\1122127876.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(test_data)
C:\Users\annza\AppData\Local\Temp\ipykernel_18016\1122127876.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(train_data)
C:\Users\annza\AppData\Local\Temp\ipykernel_18016\1122127876.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(test_data)
C:\Users\annza\AppData\Local\Temp\ipyk

In [79]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)
len(train_df), len(test_df)

(11981, 2116)

In [78]:
df.groupby(by='type_name').count()

,frame_id,type_id,obj_id,path_img,path_mask,x1,y1,x2,y2,height,width
type_name,,,,,,,,,,,
klikun,4371,4371,4371,4371,4371,4371,4371,4371,4371,4371,4371
maliy,6027,6027,6027,6027,6027,6027,6027,6027,6027,6027,6027
shipun,3699,3699,3699,3699,3699,3699,3699,3699,3699,3699,3699
